In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
import datetime
from pyspark.sql.functions import min, col, date_format
from pyspark.sql.types import StringType
from pyspark.sql.functions import sum
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.functions import first
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import min, col, datediff, current_date
from pyspark.sql import Window

In [78]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()

item = spark.read.format("csv").option("header", "true").load("LAKEHOUSE_SILVER_LAYER/items1.csv")
prod = spark.read.format("csv").option("header", "true").load("LAKEHOUSE_SILVER_LAYER/production.csv")
warehouse = spark.read.format("csv").option("header", "true").load("LAKEHOUSE_SILVER_LAYER/warehouse.csv")

In [79]:
modwarehouse = warehouse

In [80]:
#string_to_datetime = udf(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y"), TimestampType())
#modwarehouse = modwarehouse.withColumn("datetime", string_to_datetime("Registering Date"))

modwarehouse = modwarehouse.withColumn("datetime", to_timestamp("Registering Date", "dd-MM-yyyy"))
modwarehouse.columns
#modwarehouse.show()

['Entry No_',
 'Journal Batch Name',
 'Line No_',
 'Registering Date',
 'Location Code',
 'Zone Code',
 'Bin Code',
 'Description',
 'Item No_',
 'Quantity',
 'Qty_ (Base)',
 'Source Type',
 'Source Subtype',
 'Source No_',
 'Source Line No_',
 'Source Subline No_',
 'Source Document',
 'Source Code',
 'Reason Code',
 'No_ Series',
 'Bin Type Code',
 'Cubage',
 'Weight',
 'Journal Template Name',
 'Whse_ Document No_',
 'Whse_ Document Type',
 'Whse_ Document Line No_',
 'Entry Type',
 'Reference Document',
 'Reference No_',
 'User ID',
 'Variant Code',
 'Qty_ per Unit of Measure',
 'Unit of Measure Code',
 'Serial No_',
 'Lot No_',
 'Warranty Date',
 'Expiration Date',
 'Phys Invt Counting Period Code',
 'Phys Invt Counting Period Type',
 'Dedicated',
 'datetime']

In [97]:
datestep1 = modwarehouse.groupBy("Lot No_", "Bin Code", "Item No_").agg(min(col("datetime")).alias("Min_Registering_Date"))
datestep1 = datestep1.select("Lot No_", "Bin Code", "Item No_", date_format(col("Min_Registering_Date"), 'dd-MM-yyyy').alias("Min_Registering_Date"))
datestep1.show()
datestep1.printSchema()

+--------------+--------+---------------+--------------------+
|       Lot No_|Bin Code|       Item No_|Min_Registering_Date|
+--------------+--------+---------------+--------------------+
|IDS-21E2-00125|     TMG|       700536-4|          06-03-2021|
|IDS-21E2-00418|     IGS|        1000457|          10-02-2022|
|IDS-21E2-00372|     IGS|         117019|          11-04-2021|
|IDS-21E2-00564|     IGS|         112048|          20-03-2021|
|IDS-21E2-00625|     IGS|         102482|          20-01-2022|
|IDS-21E2-00560|     TMG|EII-CNS-OT-0025|          04-05-2021|
|IDS-21E2-00316|     IGS|         109456|          22-12-2022|
|IDS-21E2-00368|     IGS|         131536|          18-01-2022|
|IDS-21E2-00565|     TMG|         105581|          23-03-2021|
|IDS-21E2-00125|     IGS|         121876|          14-02-2022|
|IDS-21E2-00039|     IGS|         116912|          25-03-2021|
|IDS-21E2-00337|     TMG|         119852|          13-02-2021|
|IDS-21E2-00337|     TMG|         111742|          15-0

In [82]:
warehouse.createOrReplaceTempView("warehouse_table")
waretable = spark.table("warehouse_table")
print(type(step1))
step1temp = waretable.groupBy("Lot No_", "Bin Code", "Item No_").agg(sum("Quantity").alias("Quantity"))
lis = step1temp.select("Quantity")
lis.show()
type(lis)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+
|Quantity|
+--------+
|  1900.0|
|     0.0|
|   375.0|
|   450.0|
|   250.0|
|  1925.0|
|   250.0|
|     0.0|
|     0.0|
|   200.0|
|   425.0|
|   200.0|
|   250.0|
|     0.0|
|   200.0|
|   250.0|
|   400.0|
|   250.0|
|     0.0|
|     0.0|
+--------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [116]:
def func(lis):
    return sorted(lis.unique())[0]

udf_func = udf(func, StringType())
step1 = waretable.groupBy("Lot No_", "Bin Code", "Item No_").agg(min("Zone Code").alias("Zone Code"))
step1 = step1.join(step1temp, on=["Lot No_", "Bin Code", "Item No_"], how='inner')
step1 = step1.join(datestep1, on=["Lot No_", "Bin Code", "Item No_"], how='inner')
step1.show()

+--------------+--------+--------+---------+--------+--------------------+
|       Lot No_|Bin Code|Item No_|Zone Code|Quantity|Min_Registering_Date|
+--------------+--------+--------+---------+--------+--------------------+
|IDS-21E2-00028|     IGS| 1000457|      IGS|  1000.0|          19-05-2021|
|IDS-21E2-00028|     IGS|  102078|      IGS|   175.0|          08-08-2022|
|IDS-21E2-00028|     IGS|  102565|      IGS|   350.0|          28-01-2022|
|IDS-21E2-00028|     IGS|  102671|      IGS|   200.0|          13-12-2021|
|IDS-21E2-00028|     IGS|  104754|      IGS|     0.0|          24-02-2021|
|IDS-21E2-00028|     IGS|  105581|      IGS|  1175.0|          15-05-2021|
|IDS-21E2-00028|     IGS|  108259|      IGS|  1625.0|          15-03-2021|
|IDS-21E2-00028|     IGS|  111742|      IGS|  1200.0|          16-02-2021|
|IDS-21E2-00028|     IGS|  112048|      IGS|   825.0|          27-07-2021|
|IDS-21E2-00028|     IGS|  113014|      IGS|   250.0|          27-12-2021|
|IDS-21E2-00028|     IGS|

In [133]:
def getdays(date):
    return int((datetime.datetime.now() - datetime.datetime.strptime(date, "%d-%m-%Y")).days)
    
print(getdays("19-05-2021"))
udffunc = F.udf(getdays)
step1.printSchema()
#daystep1 = step1.withColumn("Min_Registering_Date",to_date(col("Min_Registering_Date"), 'dd-MM-yyyy'))
daystep1 = daystep1.withColumn("Days from Registration",udffunc(col("Min_Registering_Date")))
daystep1.show()
#daystep1 = daystep1.withColumn("Days", F.to_date(F.col("Min_Registering_Date").cast("string"),"%Y-%m-%d").alias("date"))
#daystep1 = step1.withColumn("Days", col("Min_Registering_Date")))

#print(current_date())
#daysstep1.show()

604
root
 |-- Lot No_: string (nullable = true)
 |-- Bin Code: string (nullable = true)
 |-- Item No_: string (nullable = true)
 |-- Zone Code: string (nullable = true)
 |-- Quantity: double (nullable = true)
 |-- Min_Registering_Date: string (nullable = true)

23/01/13 02:06:53 ERROR Executor: Exception in task 0.0 in stage 252.0 (TID 176)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 670, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 507, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 670, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 507, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 289, in read_single_udf
    f, return_type = read

In [68]:
step1.filter(col("Zone Code") == "IGS").filter(col("Item No_") == "102671").filter(col("Lot No_") == "IDS-21E2-00028").show()



+--------------+--------+--------+---------+--------+
|       Lot No_|Bin Code|Item No_|Zone Code|Quantity|
+--------------+--------+--------+---------+--------+
|IDS-21E2-00028|     IGS|  102671|      IGS|   200.0|
+--------------+--------+--------+---------+--------+



In [ ]:
df.withColumn("new_column", F.to_date(F.col("column").cast("string"),"%Y-%m-%d").alias("date"))